# Lab For Experimentation

In [1]:
import json
import warnings
from typing import Any, Literal

import numpy as np
import pandas as pd
import polars as pl
from rich.console import Console
from rich.theme import Theme

custom_theme = Theme(
    {
        "white": "#FFFFFF",  # Bright white
        "info": "#00FF00",  # Bright green
        "warning": "#FFD700",  # Bright gold
        "error": "#FF1493",  # Deep pink
        "success": "#00FFFF",  # Cyan
        "highlight": "#FF4500",  # Orange-red
    }
)
console = Console(theme=custom_theme)

# Visualization
# import matplotlib.pyplot as plt

# NumPy settings
np.set_printoptions(precision=4)

# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

# Polars settings
pl.Config.set_fmt_str_lengths(1_000)
pl.Config.set_tbl_cols(n=1_000)
pl.Config.set_tbl_rows(n=200)

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

In [2]:
def go_up_from_current_directory(*, go_up: int = 1) -> None:
    """This is used to up a number of directories.

    Params:
    -------
    go_up: int, default=1
        This indicates the number of times to go back up from the current directory.

    Returns:
    --------
    None
    """
    import os
    import sys

    CONST: str = "../"
    NUM: str = CONST * go_up

    # Goto the previous directory
    prev_directory = os.path.join(os.path.dirname(__name__), NUM)
    # Get the 'absolute path' of the previous directory
    abs_path_prev_directory = os.path.abspath(prev_directory)

    # Add the path to the System paths
    sys.path.insert(0, abs_path_prev_directory)
    print(abs_path_prev_directory)


# Demo (Prevents ruff from removing the unused module import)
name: Any
category: Literal["A", "B", "C"]
json.loads('{"name": "Bike Rental Prediction", "category": "A"}')

{'name': 'Bike Rental Prediction', 'category': 'A'}

In [3]:
go_up_from_current_directory(go_up=1)

/Users/mac/Desktop/Projects/Bike-Rental-Prediction


In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [5]:
rng = np.random.default_rng(123)
x = rng.standard_normal(size=(1_000, 10))

X_train, X_test = train_test_split(x, test_size=0.2, random_state=123)
y_train = rng.standard_normal(size=(X_train.shape[0],))
y_test = rng.standard_normal(size=(X_test.shape[0],))

params: dict[str, Any] = {
    "n_estimators": 100,
    "max_depth": 10,
}

rf_reg = RandomForestRegressor(**params)

rf_reg.fit(X_train, y_train)
# rf_reg.score(X_test, y_test)

,n_estimators,100
,criterion,'squared_error'
,max_depth,10
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [6]:
rf_reg.feature_importances_

array([0.1244, 0.0951, 0.0774, 0.1129, 0.0909, 0.1176, 0.0949, 0.0998,
       0.0889, 0.098 ])

In [7]:
import xgboost as xgb

In [8]:
from src.config import app_config
from src.exceptions import (
    CustomError,
    MLFlowConnectionError,
)
from src.ml.feature_engineering import FeatureEngineer
from src.ml.trainer import ModelTrainer

In [9]:
import httpx

port: int = 5001
url: str = f"http://localhost:{port}"


def check_mlflow(url: str, timeout: float = 2.0) -> bool:
    """
    Check MLflow endpoint accessibility and handle common httpx errors.

    Returns True if reachable (2xx), False otherwise.
    """
    try:
        resp = httpx.get(url, timeout=timeout)
        resp.raise_for_status()
        console.print("[success]MLflow is accessible")
        return True

    except httpx.HTTPStatusError as e:
        console.print(
            f"[error]MLflow returned non-2xx status: {e.response.status_code} — {e}"
        )
        return False

    except httpx.RequestError as e:
        # covers ConnectError, ReadTimeout, etc.
        console.print(f"[error]Network/connection error when contacting MLflow: {e}")
        return False

    except (MLFlowConnectionError, CustomError) as e:
        console.print(f"[error]Project-specific MLflow error: {e}")
        return False

    except Exception as e:
        console.print(f"[error]Unexpected error: {e}")
        return False


check_mlflow(url)

MLflow is accessible

True

In [10]:
fp: str = "../../../../Documents/data_dump/bike_data/database.parquet"
data: pl.DataFrame = pl.read_parquet(fp)
console.print(f"Shape: {data.shape}", style="info")

feat_eng = FeatureEngineer()
features_df: pl.DataFrame = feat_eng.create_all_features(
    data=data, config=app_config.feature_config
)
features_df.head()

Shape: (13903, 16)

season,mnth,hr,holiday,weekday,workingday,weathersit,temp,hum,is_weekend,sin_hour,cos_hour,sin_weekday,cos_weekday,cnt_lag_0hr,cnt_lag_1hr,cnt_lag_24hr,hr_lag_1hr,hr_lag_24hr,temp_lag_1hr,temp_lag_3hr,hum_lag_1hr,hum_lag_3hr,temp_rolling_mean_3hr,temp_rolling_median_3hr,temp_rolling_mean_6hr,temp_rolling_median_6hr,hum_rolling_mean_3hr,hum_rolling_median_3hr,hum_rolling_mean_6hr,hum_rolling_median_6hr,temp_plus_hum,hum_plus_hr,cnt_diff_1hr,cnt_diff_2hr,hr_diff_1hr,hr_diff_24hr,temp_diff_1hr,temp_diff_2hr,temp_diff_24hr,hum_diff_1hr,hum_diff_2hr,is_high_temp,is_high_hum,is_peak_hour,is_working_hour,is_business_hour,target
i64,i64,i64,i64,i64,i64,i64,f64,f64,i8,f64,f64,f64,f64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,f64,f64,f64,f64,f64,i8,i8,i8,i8,i8,i64
1,1,0,0,6,0,1,0.24,0.81,1,0.0,1.0,-0.781831,0.62349,16,16,16,0,0,0.24,0.24,0.81,0.81,0.226667,0.22,0.233333,0.24,0.803333,0.8,0.776667,0.775,1.05,0.81,24,16,1,0,-0.02,-0.02,0.22,-0.01,-0.01,0,0,0,0,0,40
1,1,1,0,6,0,1,0.22,0.8,1,0.258819,0.965926,-0.781831,0.62349,40,16,16,0,0,0.24,0.24,0.81,0.81,0.226667,0.22,0.233333,0.24,0.803333,0.8,0.776667,0.775,1.02,1.8,24,16,1,0,-0.02,-0.02,0.22,-0.01,-0.01,0,0,0,0,0,32
1,1,2,0,6,0,1,0.22,0.8,1,0.5,0.866025,-0.781831,0.62349,32,40,16,1,0,0.22,0.24,0.8,0.81,0.226667,0.22,0.233333,0.24,0.803333,0.8,0.776667,0.775,1.02,2.8,-8,16,1,0,0.0,-0.02,0.22,0.0,-0.01,0,0,0,0,0,13
1,1,3,0,6,0,1,0.24,0.75,1,0.707107,0.707107,-0.781831,0.62349,13,32,16,2,0,0.22,0.24,0.8,0.81,0.226667,0.22,0.233333,0.24,0.783333,0.8,0.776667,0.775,0.99,3.75,-19,-27,1,0,0.02,0.02,0.22,-0.05,-0.05,0,0,0,0,0,1
1,1,4,0,6,0,1,0.24,0.75,1,0.866025,0.5,-0.781831,0.62349,1,13,16,3,0,0.24,0.22,0.75,0.8,0.233333,0.24,0.233333,0.24,0.766667,0.75,0.776667,0.775,0.99,4.75,-12,-31,1,0,0.0,0.02,0.22,0.0,-0.05,0,0,0,0,0,1


In [11]:
from src.ml.utils import split_into_train_val_test_sets

trainer = ModelTrainer(data=features_df)

2025-10-12 22:21:11 - mlflow_tracker - [INFO] - Set MLflow tracking URI to: http://localhost:5001
2025-10-12 22:21:12 - mlflow_tracker - [INFO] - Set experiment to: bike rental (ID: 1)
2025-10-12 22:21:12 - mlflow_tracker - [INFO] - Initialized MLFlowTracker with experiment: bike rental
Shapes -> x_train: (11260, 47), y_train: (11260,), x_val: (1252, 47), y_val: (1252,), x_test: (1391, 47), y_test: (1391,)
2025-10-12 22:21:12 - trainer - [INFO] - Data preparation complete.


In [12]:
params: dict[str, Any] = {
    "objective": "reg:squarederror",
    "learning_rate": 0.1,
    "max_depth": 6,
}
data_dict = split_into_train_val_test_sets(
    data=features_df, target_col="target", test_size=0.2
)
x_train, y_train = data_dict["x_train"], data_dict["y_train"]
x_val, y_val = data_dict["x_val"], data_dict["y_val"]
dtrain = xgb.DMatrix(x_train, y_train, enable_categorical=True)
dval = xgb.DMatrix(x_val, y_val, enable_categorical=True)

# cv = xgb.cv(
#     params=params,
#     dtrain=dtrain,
#     num_boost_round=500,
#     nfold=5,
#     metrics={"rmse"},
#     early_stopping_rounds=10,
#     seed=123,
# )

# trainer._train_lightgbm(params={})
result = trainer.hyperparameter_tuning_all_models()

Shapes -> x_train: (8897, 47), y_train: (8897,), x_val: (2225, 47), y_val: (2225,), x_test: (2781, 47), y_test: (2781,)
2025-10-12 22:21:13 - trainer - [INFO] - 🚨 Starting hyperparameter tuning for all models...
2025-10-12 22:21:13 - trainer - [INFO] - 🚨 Starting hyperparameter tuning for Random Forest with 2 trials...
2025-10-12 22:21:14 - mlflow_tracker - [INFO] - Started MLflow run: 9c3647162a3f48209a7c40b04640a30f (name: run_2025-10-12T22:21:13)


[I 2025-10-12 22:21:14,033] A new study created in memory with name: no-name-d46a8234-d2fb-4d49-8b36-5cda60811467


2025-10-12 22:21:14 - mlflow_tracker - [INFO] - Started MLflow run: d252b692815d4d2b9747bc488b2c4eb8 (name: run_2025-10-12T22:21:14)
Trial 0: Mean RMSE = 50.11
🏃 View run run_2025-10-12T22:21:14 at: http://localhost:5001/#/experiments/1/runs/d252b692815d4d2b9747bc488b2c4eb8
🧪 View experiment at: http://localhost:5001/#/experiments/1
2025-10-12 22:22:26 - mlflow_tracker - [INFO] - Ended MLflow run with status: FINISHED


[I 2025-10-12 22:22:26,312] Trial 0 finished with value: 50.11 and parameters: {'n_estimators': 144, 'max_depth': 20, 'min_samples_split': 8, 'min_samples_leaf': 3, 'max_features': None, 'bootstrap': True}. Best is trial 0 with value: 50.11.


2025-10-12 22:22:26 - trainer - [INFO] - Initial trial 0 achieved value: 50.11
2025-10-12 22:22:26 - mlflow_tracker - [INFO] - Started MLflow run: d9c68c40b51c4fcda41f5a1fb5149e5d (name: run_2025-10-12T22:22:26)
Trial 1: Mean RMSE = 119.12
🏃 View run run_2025-10-12T22:22:26 at: http://localhost:5001/#/experiments/1/runs/d9c68c40b51c4fcda41f5a1fb5149e5d
🧪 View experiment at: http://localhost:5001/#/experiments/1
2025-10-12 22:22:57 - mlflow_tracker - [INFO] - Ended MLflow run with status: FINISHED


...

2025-10-12 22:24:56 - mlflow_tracker - [INFO] - Started MLflow run: 962df5ef0e6f4679b3fc0078123ffca3 (name: run_2025-10-12T22:24:56)
Trial 0: Mean RMSE = 56.88
🏃 View run run_2025-10-12T22:24:56 at: http://localhost:5001/#/experiments/1/runs/962df5ef0e6f4679b3fc0078123ffca3
🧪 View experiment at: http://localhost:5001/#/experiments/1
2025-10-12 22:24:57 - mlflow_tracker - [INFO] - Ended MLflow run with status: FINISHED


[I 2025-10-12 22:24:57,191] Trial 0 finished with value: 56.88 and parameters: {'reg_lambda': 0.03148911647956861, 'reg_alpha': 6.3512210106407005, 'learning_rate': 0.17524101118128144, 'num_leaves': 188, 'max_depth': 5, 'min_child_samples': 12, 'num_boost_round': 210, 'early_stopping_rounds': 88}. Best is trial 0 with value: 56.88.


2025-10-12 22:24:57 - trainer - [INFO] - Initial trial 0 achieved value: 56.88
2025-10-12 22:24:57 - mlflow_tracker - [INFO] - Started MLflow run: 015fba07b6cc4e1984a4b08d7416b45c (name: run_2025-10-12T22:24:57)
Trial 1: Mean RMSE = 58.63
🏃 View run run_2025-10-12T22:24:57 at: http://localhost:5001/#/experiments/1/runs/015fba07b6cc4e1984a4b08d7416b45c
🧪 View experiment at: http://localhost:5001/#/experiments/1
2025-10-12 22:25:24 - mlflow_tracker - [INFO] - Ended MLflow run with status: FINISHED


[I 2025-10-12 22:25:24,276] Trial 1 finished with value: 58.63 and parameters: {'reg_lambda': 0.25378155082656645, 'reg_alpha': 0.6796578090758157, 'learning_rate': 0.01083858126934475, 'num_leaves': 292, 'max_depth': 13, 'min_child_samples': 14, 'num_boost_round': 445, 'early_stopping_rounds': 26}. Best is trial 0 with value: 56.88.


Best trial: 0 | Value: 56.88
Training until validation scores don't improve for 88 rounds
Did not meet early stopping. Best iteration is:
[151]	validation's l2: 3235.54
2025-10-12 22:25:25 - mlflow_tracker - [INFO] - ✅ Successfully logged ModelType.LIGHTGBM model and metadata
🏃 View run run_2025-10-12T22:24:56 at: http://localhost:5001/#/experiments/1/runs/4a1037cdd9fa440aa7836d8fa7a7833f
🧪 View experiment at: http://localhost:5001/#/experiments/1
2025-10-12 22:25:25 - mlflow_tracker - [INFO] - Ended MLflow run with status: FINISHED
2025-10-12 22:25:25 - trainer - [INFO] - ✅ Hyperparameter tuning completed successfully.


In [ ]:
# trainer.hyperparameter_tuning_all_models()

In [ ]:
result = trainer._hyperparameter_tuning_xgboost()
# result = trainer._hyperparameter_tuning_lightgbm()

result

In [13]:
import mlflow

try:
    mlflow.end_run()
except Exception as e:
    print(e)

trainer.mlflow_tracker.load_model_artifact(
    run_id=result["run_id"],
    model_name=ModelType.XGBOOST.value,
    artifact_subpath="models",
)

In [14]:
from src.exp_tracking.model_loader import (
    load_best_model,
)

In [17]:
# console.print(result, style="highlight")

# Load the best model from the experiment
best_model_artifacts = load_best_model(
    experiment_name=app_config.experiment_config.experiment_name
)

if best_model_artifacts:
    console.print(
        f"[success]Loaded best model: {best_model_artifacts.get('metadata', {}).get('model_type', 'Unknown')}",
        style="success",
    )
else:
    console.print("[warning]No best model found!", style="warning")

2025-10-12 22:27:06 - model_loader - [INFO] - Best run: 9c3647162a3f48209a7c40b04640a30f with best_rmse=50.11
2025-10-12 22:27:06 - mlflow_tracker - [INFO] - Set MLflow tracking URI to: http://localhost:5001
2025-10-12 22:27:06 - mlflow_tracker - [INFO] - Set experiment to: bike rental (ID: 1)
2025-10-12 22:27:06 - mlflow_tracker - [INFO] - Initialized MLFlowTracker with experiment: bike rental


2025-10-12 22:27:10 - mlflow_tracker - [INFO] - Detected model file: 'ModelType.RANDOM_FOREST_2025-10-12T22:24:50_model.pkl' with extension 'pkl'
2025-10-12 22:27:10 - mlflow_tracker - [INFO] - ✅ Successfully loaded ModelType.RANDOM_FOREST model from run 9c3647162a3f48209a7c40b04640a30f
2025-10-12 22:27:10 - model_loader - [INFO] - ✅ Loaded best ModelType.RANDOM_FOREST model


Loaded best model: ModelType.RANDOM_FOREST

In [16]:
# Display the loaded best model artifacts
console.print(best_model_artifacts)
best_model_artifacts.keys()

{
    'model': RandomForestRegressor(max_depth=20, max_features=None, min_samples_leaf=3,
                      min_samples_split=8, n_estimators=144, random_state=42),
    'metadata': {
        'model_type': 'ModelType.RANDOM_FOREST',
        'framework': 'sklearn.ensemble._forest',
        'class': 'RandomForestRegressor',
        'save_format': 'pkl',
        'timestamp': '2025-10-12T22:24:50',
        'input_example': {
            'num_rows': 13903,
            'num_cols': 48,
            'dtypes': {
                'season': 'Int64',
                'mnth': 'Int64',
                'hr': 'Int64',
                'holiday': 'Int64',
                'weekday': 'Int64',
                'workingday': 'Int64',
                'weathersit': 'Int64',
                'temp': 'Float64',
                'hum': 'Float64',
                'is_weekend': 'Int8',
                'sin_hour': 'Float64',
                'cos_hour': 'Float64',
                'sin_weekday': 'Float64',
                'cos_weekday': 'Float64',
                'cnt_lag_0hr': 'Int64',
                'cnt_lag_1hr': 'Int64',
                'cnt_lag_24hr': 'Int64',
                'hr_lag_1hr': 'Int64',
                'hr_lag_24hr': 'Int64',
                'temp_lag_1hr': 'Float64',
                'temp_lag_3hr': 'Float64',
                'hum_lag_1hr': 'Float64',
                'hum_lag_3hr': 'Float64',
                'temp_rolling_mean_3hr': 'Float64',
                'temp_rolling_median_3hr': 'Float64',
                'temp_rolling_mean_6hr': 'Float64',
                'temp_rolling_median_6hr': 'Float64',
                'hum_rolling_mean_3hr': 'Float64',
                'hum_rolling_median_3hr': 'Float64',
                'hum_rolling_mean_6hr': 'Float64',
                'hum_rolling_median_6hr': 'Float64',
                'temp_plus_hum': 'Float64',
                'hum_plus_hr': 'Float64',
                'cnt_diff_1hr': 'Int64',
                'cnt_diff_2hr': 'Int64',
                'hr_diff_1hr': 'Int64',
                'hr_diff_24hr': 'Int64',
                'temp_diff_1hr': 'Float64',
                'temp_diff_2hr': 'Float64',
                'temp_diff_24hr': 'Float64',
                'hum_diff_1hr': 'Float64',
                'hum_diff_2hr': 'Float64',
                'is_high_temp': 'Int8',
                'is_high_hum': 'Int8',
                'is_peak_hour': 'Int8',
                'is_working_hour': 'Int8',
                'is_business_hour': 'Int8',
                'target': 'Int64'
            }
        },
        'model_params': {
            'bootstrap': True,
            'ccp_alpha': 0.0,
            'criterion': 'squared_error',
            'max_depth': 20,
            'max_features': None,
            'max_leaf_nodes': None,
            'max_samples': None,
            'min_impurity_decrease': 0.0,
            'min_samples_leaf': 3,
            'min_samples_split': 8,
            'min_weight_fraction_leaf': 0.0,
            'monotonic_cst': None,
            'n_estimators': 144,
            'n_jobs': None,
            'oob_score': False,
            'random_state': 42,
            'verbose': 0,
            'warm_start': False
        }
    },
    'input_example': [
        {
            'season': 1,
            'mnth': 1,
            'hr': 0,
            'holiday': 0,
            'weekday': 6,
            'workingday': 0,
            'weathersit': 1,
            'temp': 0.24,
            'hum': 0.81,
            'is_weekend': 1,
            'sin_hour': 0.0,
            'cos_hour': 1.0,
            'sin_weekday': -0.7818314824680299,
            'cos_weekday': 0.6234898018587334,
            'cnt_lag_0hr': 16,
            'cnt_lag_1hr': 16,
            'cnt_lag_24hr': 16,
            'hr_lag_1hr': 0,
            'hr_lag_24hr': 0,
            'temp_lag_1hr': 0.24,
            'temp_lag_3hr': 0.24,
            'hum_lag_1hr': 0.81,
            'hum_lag_3hr': 0.81,
            'temp_rolling_mean_3hr': 0.22666666666666666,
    

dict_keys(['model', 'metadata', 'input_example', 'model_uri', 'run_id', 'model_name'])

In [21]:
trainer.input_example.head()

season,mnth,hr,holiday,weekday,workingday,weathersit,temp,hum,is_weekend,sin_hour,cos_hour,sin_weekday,cos_weekday,cnt_lag_0hr,cnt_lag_1hr,cnt_lag_24hr,hr_lag_1hr,hr_lag_24hr,temp_lag_1hr,temp_lag_3hr,hum_lag_1hr,hum_lag_3hr,temp_rolling_mean_3hr,temp_rolling_median_3hr,temp_rolling_mean_6hr,temp_rolling_median_6hr,hum_rolling_mean_3hr,hum_rolling_median_3hr,hum_rolling_mean_6hr,hum_rolling_median_6hr,temp_plus_hum,hum_plus_hr,cnt_diff_1hr,cnt_diff_2hr,hr_diff_1hr,hr_diff_24hr,temp_diff_1hr,temp_diff_2hr,temp_diff_24hr,hum_diff_1hr,hum_diff_2hr,is_high_temp,is_high_hum,is_peak_hour,is_working_hour,is_business_hour,target
i64,i64,i64,i64,i64,i64,i64,f64,f64,i8,f64,f64,f64,f64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,f64,f64,f64,f64,f64,i8,i8,i8,i8,i8,i64
1,1,0,0,6,0,1,0.24,0.81,1,0.0,1.0,-0.781831,0.62349,16,16,16,0,0,0.24,0.24,0.81,0.81,0.226667,0.22,0.233333,0.24,0.803333,0.8,0.776667,0.775,1.05,0.81,24,16,1,0,-0.02,-0.02,0.22,-0.01,-0.01,0,0,0,0,0,40
1,1,1,0,6,0,1,0.22,0.8,1,0.258819,0.965926,-0.781831,0.62349,40,16,16,0,0,0.24,0.24,0.81,0.81,0.226667,0.22,0.233333,0.24,0.803333,0.8,0.776667,0.775,1.02,1.8,24,16,1,0,-0.02,-0.02,0.22,-0.01,-0.01,0,0,0,0,0,32
1,1,2,0,6,0,1,0.22,0.8,1,0.5,0.866025,-0.781831,0.62349,32,40,16,1,0,0.22,0.24,0.8,0.81,0.226667,0.22,0.233333,0.24,0.803333,0.8,0.776667,0.775,1.02,2.8,-8,16,1,0,0.0,-0.02,0.22,0.0,-0.01,0,0,0,0,0,13
1,1,3,0,6,0,1,0.24,0.75,1,0.707107,0.707107,-0.781831,0.62349,13,32,16,2,0,0.22,0.24,0.8,0.81,0.226667,0.22,0.233333,0.24,0.783333,0.8,0.776667,0.775,0.99,3.75,-19,-27,1,0,0.02,0.02,0.22,-0.05,-0.05,0,0,0,0,0,1
1,1,4,0,6,0,1,0.24,0.75,1,0.866025,0.5,-0.781831,0.62349,1,13,16,3,0,0.24,0.22,0.75,0.8,0.233333,0.24,0.233333,0.24,0.766667,0.75,0.776667,0.775,0.99,4.75,-12,-31,1,0,0.0,0.02,0.22,0.0,-0.05,0,0,0,0,0,1


In [34]:
trainer.mlflow_tracker.register_model(
    run_id=best_model_artifacts.get("run_id"),
    model=best_model_artifacts.get("model"),
    model_name=best_model_artifacts.get("model_name"),
    input_example=trainer.input_example,
)

2025/10/12 22:42:31 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/12 22:42:35 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Registered model 'ModelType.RANDOM_FOREST_best' already exists. Creating a new version of this model...
2025/10/12 22:42:36 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ModelType.RANDOM_FOREST_best, version 15


2025-10-12 22:42:36 - mlflow_tracker - [INFO] - Registered model: models:/m-60af970b7aba4248b75cf9b4f9ee4ddc | version: 15


Created version '15' of model 'ModelType.RANDOM_FOREST_best'.


🏃 View run run_2025-10-12T22:21:13 at: http://localhost:5001/#/experiments/1/runs/9c3647162a3f48209a7c40b04640a30f
🧪 View experiment at: http://localhost:5001/#/experiments/1


In [ ]:
best_model_artifacts["metadata"].keys()

In [28]:
# Extract model information
model = best_model_artifacts["model"]
model_name = best_model_artifacts["model_name"]
run_id = best_model_artifacts["run_id"]
metadata = best_model_artifacts["metadata"]
input_example = best_model_artifacts.get("input_example")

# Convert input example to appropriate format if available
input_example_df = None
if input_example:
    import pandas as pd

    input_example_df = pd.DataFrame(input_example[:1]).drop(columns=["target"])

# Prepare tags
tags: dict[str, Any] = {
    "experiment_name": app_config.experiment_config.experiment_name,
    "model_type": metadata.get("model_type", "Unknown"),
    "timestamp": metadata.get("timestamp", "Unknown"),
}

# Log the model using MLflow's native format for model registry
# This creates a proper MLmodel file that can be registered
try:
    with mlflow.start_run(run_id=run_id):
        # Set tags for the run
        mlflow.set_tags(tags)

        # Determine model type and log appropriately
        if "XGBOOST" in model_name:
            model_info = mlflow.xgboost.log_model(
                xgb_model=model,
                artifact_path="registered_model",
                registered_model_name=f"{model_name}_best",
                input_example=input_example_df,
            )
        elif "LIGHTGBM" in model_name:
            model_info = mlflow.lightgbm.log_model(
                lgb_model=model,
                artifact_path="registered_model",
                registered_model_name=f"{model_name}_best",
                input_example=input_example_df,
            )
        elif "RANDOM_FOREST" in model_name:
            model_info = mlflow.sklearn.log_model(
                sk_model=model,
                artifact_path="registered_model",
                registered_model_name=f"{model_name}_best",
                input_example=input_example_df,
                metadata=tags,
            )
        else:
            raise ValueError(f"Unsupported model type: {model_name}")


except Exception as e:
    console.print(f"[error]Error registering model: {e}", style="error")

2025/10/12 22:40:28 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/12 22:40:36 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Registered model 'ModelType.RANDOM_FOREST_best' already exists. Creating a new version of this model...
2025/10/12 22:40:39 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ModelType.RANDOM_FOREST_best, version 14
Created version '14' of model 'ModelType.RANDOM_FOREST_best'.


🏃 View run run_2025-10-12T22:21:13 at: http://localhost:5001/#/experiments/1/runs/9c3647162a3f48209a7c40b04640a30f
🧪 View experiment at: http://localhost:5001/#/experiments/1


In [33]:
model_info.registered_model_version

'14'